# Run multiple robots for one day

Run an experiment with multiple robots exploring an environment during the course of one day. 

The specification of the experiment is fully determined by the exp/run of type nrobot1day. This exp/run will refer to other exp/runs to specify the environment, the estimator and the scoring function. It will also list the names of the robots, their policies and other information. 

In [5]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "WaterBerryFarms"

import pathlib
from pprint import pprint

from wbf_simulate import run_nrobot1day

In [6]:
# the experiment for the robot run
experiment = "nrobot1day"
# run = "two-random-robots"
# run = "two-communicating-robots"
# run = "mrmr-3random"
# run = "mrmr-3lawnmowers"
run = "mrmr-1pioneer-2contractors"
exp = Config().get_experiment(experiment, run)
pprint(exp)

***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\WaterBerryFarms\settings\experiment-config\LotziYoga\nrobot1day\mrmr-1pioneer-2contractors_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: nrobot1day/mrmr-1pioneer-2contractors successfully loaded
Experiment: {'velocity': 1, 'timesteps-per-day': 1000, 'time-start-environment': 25, 'im_resolution': 10, 'experiment_name': 'nrobot1day', 'name': '1 pioneer 2 contractors', 'exp_environment': 'miniberry-100', 'exp_estimator': 'adaptive-disk', 'exp_score': 'weighted_asymmetric', 'robots': [{'name': 'robot-pioneer', 'exp-policy': 'generic', 'exp-policy-extra-parameters': {'policy-name': 'mrmr-pioneer', 'policy-code': 'MRMR_Pioneer', 'seed': 1, 'budget': 1000}}, {'name': 'robot-contr-1', 'exp-policy': 'generic', 'exp-policy-extra-parameters': {'policy-name': 'mrmr-contractor-1', 'policy-code': 'MRMR_Contractor', 'seed': 2, 'budget': 1000}}, {'name': 'robot-contr-2', 'exp

In [7]:
run_nrobot1day(exp)

***ExpRun**: Experiment default config C:\Users\lboloni\Documents\Code\_Checkouts\WBF\WaterberryFarms\experiment_configs\environment\_environment.yaml was empty, ok.
***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\WaterBerryFarms\settings\experiment-config\LotziYoga\environment\miniberry-100_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: environment/miniberry-100 successfully loaded
Experiment: {'experiment_name': 'environment', 'typename': 'Miniberry-100', 'precompute-time': 100, 'run_name': 'miniberry-100', 'exp_run_sys_indep_file': 'C:\\Users\\lboloni\\Documents\\Code\\_Checkouts\\WBF\\WaterberryFarms\\experiment_configs\\environment\\miniberry-100.yaml', 'data_dir': 'C:\\Users\\lboloni\\Documents\\Code\\_TempData\\WBF\\ExperimentData\\environment\\miniberry-100', 'subrun_name': None, 'time_started': '2025-04-27 12:59:39'}
***ExpRun**: Experiment default config C:\Users\lboloni\Documents\Code\_Checkouts\W

INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:PrecalculatedEnvironment at timestamp 25
INFO:root:Loading from gz C:\Users\lboloni\Documents\Code\_TempData\WBF\ExperimentData\environment\miniberry-100\precalc_tylcv\env_value_00025.gz
INFO:root:Loading from gz C:\Users\lboloni\Documents\Code\_TempData\WBF\ExperimentData\environment\miniberry-100\precalc_tylcv\env_value_00025.gz done
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:PrecalculatedEnvironment at timestamp 25
INFO:root:Loading from gz C:\Users\lboloni\Documents\Code\_TempData\WBF\ExperimentData\environment\miniberry-100\precalc_ccr\env_value_00025.gz
INFO:root:Loading from gz C:\Users\lboloni\Documents\Code\_TempData\WBF\ExperimentData\environment\miniberry-100\precalc_ccr\env_value_00025.gz done
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:PrecalculatedEnvironment at timestamp 25
INFO:root:Loading from gz

policy-name
policy-code
seed
budget
[{'exp-policy': Experiment: {'experiment_name': 'policy', 'policy-name': 'mrmr-pioneer', 'policy-code': 'MRMR_Pioneer', 'run_name': 'generic', 'exp_run_sys_indep_file': 'C:\\Users\\lboloni\\Documents\\Code\\_Checkouts\\WBF\\WaterberryFarms\\experiment_configs\\policy\\generic.yaml', 'data_dir': 'C:\\Users\\lboloni\\Documents\\Code\\_TempData\\WBF\\ExperimentData\\policy\\generic', 'subrun_name': None, 'time_started': '2025-04-27 12:59:39', 'seed': 1, 'budget': 1000},
  'name': 'robot-pioneer'},
 {'exp-policy': Experiment: {'experiment_name': 'policy', 'policy-name': 'mrmr-contractor-1', 'policy-code': 'MRMR_Contractor', 'run_name': 'generic', 'exp_run_sys_indep_file': 'C:\\Users\\lboloni\\Documents\\Code\\_Checkouts\\WBF\\WaterberryFarms\\experiment_configs\\policy\\generic.yaml', 'data_dir': 'C:\\Users\\lboloni\\Documents\\Code\\_TempData\\WBF\\ExperimentData\\policy\\generic', 'subrun_name': None, 'time_started': '2025-04-27 12:59:39', 'seed': 2, '

IndexError: index 101 is out of bounds for axis 1 with size 101